In [ ]:
!nvidia-smi

In [ ]:
# !pip install -q torch==1.8.1+cu111 torchvision==0.9.1+cu111 -f https://download.pytorch.org/whl/torch_stable.html
# !conda install pytorch torchvision torchaudio cudatoolkit=11.1 -c pytorch -c nvidia

!pip install timm

In [ ]:
# !pip uninstall torchtext -y
# !pip uninstall torchaudio -y

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd
import os
import timm
import random
import time
from collections import OrderedDict
from torch.cuda import amp
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms as T
from torchvision.io import read_image
from tqdm import tqdm
print(torch.__version__)

In [ ]:
!pwd

In [ ]:

@torch.no_grad()
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)
    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.reshape(1, -1).expand_as(pred))
    return [correct[:k].reshape(-1).float().sum(0) * 100. / batch_size for k in topk]


def set_debug_apis(state: bool = False):
    torch.autograd.profiler.profile(enabled=state)
    torch.autograd.profiler.emit_nvtx(enabled=state)
    torch.autograd.set_detect_anomaly(mode=state)


def seed_everything(seed):
    """
    Makes code deterministic using a given seed.
    Internally sets all seeds of torch, numpy and random.
    """
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True


def print_size_of_model(model):
    torch.save(model.state_dict(), "temp.p")
    print("Size (MB):", os.path.getsize("temp.p") / 1e6)
    os.remove("temp.p")


class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
class RetinopathyDataset(Dataset):
    def __init__(self, image_dir, csv_file, transforms=None):
        self.data = pd.read_csv(csv_file)
        self.transforms = transforms
        self.image_dir = image_dir

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # img_name = os.path.join('../input/aptos2019-blindness-detection/train_images',
        #                         self.data.loc[idx, 'id_code'] + '.png')

        img_name = os.path.join(self.image_dir, self.data.loc[idx, 'id_code'] + '.png')

        tensor_image = read_image(img_name)
        label = torch.tensor(self.data.loc[idx, 'diagnosis'], dtype=torch.long)

        if self.transforms is not None:
            tensor_image = self.transforms(tensor_image)

        return (tensor_image, label)

In [ ]:
print(torch.cuda.is_available())

In [ ]:
def train_step(model: nn.Module, train_loader, criterion,
               device: str, optimizer,
               scheduler=None, num_batches: int = None,
               log_interval: int = 100,
               scaler=None,):
    """
    Performs one step of training. Calculates loss, forward pass, computes gradient and returns metrics.
    Args:
        model : A pytorch CNN Model.
        train_loader : Train loader.
        criterion : Loss function to be optimized.
        device : "cuda" or "cpu"
        optimizer : Torch optimizer to train.
        scheduler : Learning rate scheduler.
        num_batches : (optional) Integer To limit training to certain number of batches.
        log_interval : (optional) Defualt 100. Integer to Log after specified batch ids in every batch.
        scaler: (optional)  Pass torch.cuda.amp.GradScaler() for fp16 precision Training.
    """

    model = model.to(device)
    start_train_step = time.time()
    metrics = OrderedDict()
    model.train()
    last_idx = len(train_loader) - 1
    batch_time_m = AverageMeter()
    # data_time_m = AverageMeter()
    losses_m = AverageMeter()
    top1_m = AverageMeter()
    top5_m = AverageMeter()
    cnt = 0
    batch_start = time.time()
    # num_updates = epoch * len(loader)

    for batch_idx, (inputs, target) in enumerate(train_loader):
        last_batch = batch_idx == last_idx
        # data_time_m.update(time.time() - batch_start)
        inputs = inputs.to(device)
        target = target.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        if scaler is not None:
            with amp.autocast():
                output = model(inputs)
                loss = criterion(output, target)
                # Scale the loss using Grad Scaler
            scaler.scale(loss).backward()
            # Step using scaler.step()
            scaler.step(optimizer)
            # Update for next iteration
            scaler.update()

        else:
            output = model(inputs)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

        if scheduler is not None:
            scheduler.step()

        cnt += 1
        acc1, acc5 = accuracy(output, target, topk=(1, 5))

        top1_m.update(acc1.item(), output.size(0))
        top5_m.update(acc5.item(), output.size(0))
        losses_m.update(loss.item(), inputs.size(0))

        batch_time_m.update(time.time() - batch_start)
        batch_start = time.time()
        if last_batch or batch_idx % log_interval == 0:  # If we reach the log intervel
            print(
                "Batch Train Time: {batch_time.val:.3f} ({batch_time.avg:.3f})  "
                "Loss: {loss.val:>7.4f} ({loss.avg:>6.4f})  "
                "Top 1 Accuracy: {top1.val:>7.4f} ({top1.avg:>7.4f})  "
                "Top 5 Accuracy: {top5.val:>7.4f} ({top5.avg:>7.4f})".format(
                    batch_time=batch_time_m, loss=losses_m, top1=top1_m, top5=top5_m))

        if num_batches is not None:
            if cnt >= num_batches:
                end_train_step = time.time()
                metrics["loss"] = losses_m.avg
                metrics["top1"] = top1_m.avg
                metrics["top5"] = top5_m.avg
                print(f"Done till {num_batches} train batches")
                print(f"Time taken for train step = {end_train_step - start_train_step} sec")
                return metrics

    metrics["loss"] = losses_m.avg
    metrics["top1"] = top1_m.avg
    metrics["top5"] = top5_m.avg
    end_train_step = time.time()
    print(f"Time taken for train step = {end_train_step - start_train_step} sec")
    return metrics

In [ ]:
@torch.no_grad()
def val_step(model: nn.Module, val_loader, criterion,
             device: str, num_batches=None,
             log_interval: int = 100):

    """
    Performs one step of validation. Calculates loss, forward pass and returns metrics.
    Args:
        model : A pytorch CNN Model.
        val_loader : Validation loader.
        criterion : Loss function to be optimized.
        device : "cuda" or "cpu"
        num_batches : (optional) Integer To limit validation to certain number of batches.
        log_interval : (optional) Defualt 100. Integer to Log after specified batch ids in every batch.
    """

    model = model.to(device)
    start_val_step = time.time()
    last_idx = len(val_loader) - 1
    batch_time_m = AverageMeter()
    # data_time_m = AverageMeter()
    losses_m = AverageMeter()
    top1_m = AverageMeter()
    top5_m = AverageMeter()
    cnt = 0
    model.eval()
    batch_start = time.time()
    metrics = OrderedDict()

    for batch_idx, (inputs, target) in enumerate(val_loader):
        last_batch = batch_idx == last_idx
        inputs = inputs.to(device)
        target = target.to(device)

        output = model(inputs)
        loss = criterion(output, target)
        cnt += 1
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        reduced_loss = loss.data

        losses_m.update(reduced_loss.item(), inputs.size(0))
        top1_m.update(acc1.item(), output.size(0))
        top5_m.update(acc5.item(), output.size(0))
        batch_time_m.update(time.time() - batch_start)

        batch_start = time.time()

        if (last_batch or batch_idx % log_interval == 0):  # If we reach the log intervel
            print(
                "Batch Inference Time: {batch_time.val:.3f} ({batch_time.avg:.3f})  "
                "Loss: {loss.val:>7.4f} ({loss.avg:>6.4f})  "
                "Top 1 Accuracy: {top1.val:>7.4f} ({top1.avg:>7.4f})  "
                "Top 5 Accuracy: {top5.val:>7.4f} ({top5.avg:>7.4f})".format(
                    batch_time=batch_time_m, loss=losses_m, top1=top1_m, top5=top5_m))

        if num_batches is not None:
            if cnt >= num_batches:
                end_val_step = time.time()
                metrics["loss"] = losses_m.avg
                metrics["top1"] = top1_m.avg
                metrics["top5"] = top5_m.avg
                print(f"Done till {num_batches} validation batches")
                print(f"Time taken for validation step = {end_val_step - start_val_step} sec")
                return metrics

    metrics["loss"] = losses_m.avg
    metrics["top1"] = top1_m.avg
    metrics["top5"] = top5_m.avg
    print("Finished the validation epoch")

    end_val_step = time.time()
    print(f"Time taken for validation step = {end_val_step - start_val_step} sec")
    return metrics


In [ ]:
DATA_DIR = "../input/aptos2019-blindness-detection/"
TRAIN_DIR = "../input/aptos2019-blindness-detection/train_images"
CSV_PATH = "../input/aptos2019-blindness-detection/train.csv"
MODEL_PATH = "."


TRAIN_SPLIT = 0.8
# Automaticllay
# VAL_SPLIT = 0.2

LEARNING_RATE = 1e-4
TRAIN_BATCH_SIZE = 32
# TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 32
# VALID_BATCH_SIZE = 16

NUM_WORKERS = 2
EPOCHS = 15

IMG_WIDTH = 768
IMG_HEIGHT = 768

MODEL_NAME = "mobilenetv3_large_100"
# MODEL_NAME = "resnet_50"


MODEL_SAVE = MODEL_PATH + MODEL_NAME
USE_AMP = True



In [ ]:
seed_everything(42)
set_debug_apis(False)

train_trasforms = T.Compose([
    T.ConvertImageDtype(torch.float32),
    T.Resize((IMG_WIDTH, IMG_HEIGHT)),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

full_dataset = RetinopathyDataset(TRAIN_DIR, CSV_PATH, transforms=train_trasforms)

train_size = int(TRAIN_SPLIT * len(full_dataset))
test_size = len(full_dataset) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])

train_loader = DataLoader(full_dataset, batch_size=TRAIN_BATCH_SIZE,
                          shuffle=False, num_workers=NUM_WORKERS, drop_last=True, pin_memory=False)

val_loader = DataLoader(val_dataset, batch_size=VALID_BATCH_SIZE, shuffle=False,
                        num_workers=NUM_WORKERS, drop_last=True, pin_memory=False)

# for batch_idx, (inputs, target) in enumerate(train_loader):
#     print(batch_idx)
#     # print(inputs)
#     print(target)
#     break

In [ ]:
model = timm.create_model(MODEL_NAME, pretrained=True, num_classes=5)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

if USE_AMP:
    from torch.cuda import amp
    scaler = amp.GradScaler()

In [ ]:
train_loss = []
train_top1_acc = []
val_loss = []
val_top1_acc = []

for epoch in tqdm(range(EPOCHS)):
    train_metrics = train_step(model, train_loader, criterion, device, optimizer, scaler=scaler)
    train_loss.append(train_metrics["loss"])
    print(f"Training loss = {train_metrics['loss']}")
    train_top1_acc.append(train_metrics["top1"])

    val_metrics = val_step(model, val_loader, criterion, device)
    val_loss.append(val_metrics["loss"])
    print(f"Validation loss = {val_metrics['loss']}")
    val_top1_acc.append(val_metrics["top1"])
    
    torch.save(model.state_dict(), f"{MODEL_NAME}_{epoch}" + ".pt")

In [ ]:
import matplotlib.pyplot as plt

plt.plot(train_loss)
plt.title(f"Training Results for {MODEL_NAME}")
plt.xlabel("Epochs")
plt.ylabel("Train Loss")
plt.show()


plt.plot(train_top1_acc)
plt.title(f"Training Results for {MODEL_NAME}")
plt.xlabel("Epochs")
plt.ylabel("Training Top1 Accuracy")
plt.show()


plt.plot(val_loss)
plt.title(f"Validation Results for {MODEL_NAME}")
plt.xlabel("Epochs")
plt.ylabel("Validation Loss")
plt.show()


plt.plot(val_top1_acc)
plt.title(f"Validation Results for {MODEL_NAME}")
plt.xlabel("Epochs")
plt.ylabel("Validation Top1 Accuracy")
plt.show()

